In [17]:
import cv2
import numpy as np
import os
import time
from Transbot_Lib import Transbot

In [18]:
list_file = ['orange', 'apple', 'donut']

def arm_speed(speed):
    time.sleep(0.2)

try:
    bot=Transbot()
except AttributeError as e:
    print(f"Erorr init Transbot:{e}")


file_path= os.path.expanduser("~/transbot_ws/src/transbot_visual/detection/object_detection_coco.txt")

# 파일을 열어 클래스 이름 읽기
with open(file_path, 'r') as f:
    classes = f.read().split('\n')


# Display different colors for different targets
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Load the DNNN image model
model = cv2.dnn.readNetFromTensorflow('frozen_inference_graph.pb', config='ssd_mobilenet_v2_coco.txt', framework='TensorFlow')




Serial Opened! Baudrate=115200


NameError: name 'cv2' is not defined

In [13]:
# Initialize the robot arm
def control_robot_arm(position):
    servo_id1 = 7
    servo_id2 = 8
    servo_id3 = 9

    if position == "up":
        bot.set_uart_servo_angle(servo_id2, 170)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 160)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 30)
    
    elif position == "mid_up":
        bot.set_uart_servo_angle(servo_id2, 170)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 140)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 30)

    elif position == "mid_down":
        bot.set_uart_servo_angle(servo_id2, 170)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 125)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 30)
    
    elif position == "down":
        bot.set_uart_servo_angle(servo_id2, 190)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 110)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 30)

    elif position == "center":
        bot.set_uart_servo_angle(servo_id2, 30)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 135)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 31)
    
    elif position == "center_grab":
        bot.set_uart_servo_angle(servo_id2, 33)
        arm_speed()
        bot.set_uart_servo_angle(servo_id1, 225)
        arm_speed()
        bot.set_uart_servo_angle(servo_id3, 180)

    elif position == "grab":
        bot.set_uart_servo_angle(servo_id3, 180)

        


In [15]:
def Target_Detection_with_Robot_Arm(image):
    image_height, image_width, _ = image.shape

    #create blob from image
    blob = cv2.dnn.blobFromImage(image, size=(300, 300), mean=(104, 117,123),swapRB=True)
    model.setInput(blob)
    output = model
    detected_objects = []

    box_x = 0
    box_y = 0
    box_height = 0
    box_width = 0

    # Iterate throug eah test
    for detection in output[0, 0, :, :]:
        confidence = detection[2]

        if confidence > 0.4:
            class_id = detection[1]
            class_name = classes[int(class_id) - 1]
            color = COLORS[int(class_id)]
            # Get the coordinates of the object
            box_x = detection[3] * image_width
            box_y = detection[4] * image_height
            box_width = detection[5] * image_width
            box_height = detection[6] * image_height
        
        # Draw a rectangle and class name as before
        cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
        cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Add detected object information to the list
        detected_objects.append(
            {class_name: class_name, 'box_x': box_x, 'box_y': box_y, 'box_width': box_width, 'box_height': box_height})


    # using robot arm to grab the target
    if detected_objects:
        target_object = detected_objects[0]
        target_x = target_object['box_x']
        target_y = target_object['box_y']
        print(f"Target Object Location: {target_x}, {target_y}")
        class_name = target_object['class_name']

        if class_name == 'orange':
            while True:

                while target_y <120:
                    control_robot_arm("up")
                    break

                while 120 < target_y < 160:
                    control_robot_arm("mid_up")
                    break   
                
                while 170 < target_y < 230:
                    control_robot_arm("mid_down")
                    break

                while target_y > 230:
                    control_robot_arm("down")
                    break
            
                break

        time.sleep(1)
        control_robot_arm("grab")
        time.sleep(2)
        control_robot_arm("center_grab")
        time.sleep(1)
    
    else:
        control_robot_arm("center")

    return image





In [16]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    detected_frame = Target_Detection_with_Robot_Arm(frame)

    if detected_frame is not None:
        cv2.imshow('frame', detected_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined